In [1]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta
import sys

# --- Database Connection Details ---
DB_HOST = "localhost"
DB_USER = "retail_user"
DB_PASSWORD = "StrongPassword!23"  # Use the password you set
DB_NAME = "retail_mart"

# --- Connect to MySQL ---
try:
    mydb = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )
    mycursor = mydb.cursor()
    print("Database connected successfully.")
    sys.stdout.flush()
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit()

fake = Faker()

# --- 1. Populate 'Supplier' table ---
print("Populating Supplier table...")
sys.stdout.flush()
num_suppliers = 10
sql_supplier = "INSERT INTO Supplier (Supplier_name, Contact_person, Supplier_phone, Supplier_email, Supplier_address) VALUES (%s, %s, %s, %s, %s)"
for i in range(num_suppliers):
    supplier_name = fake.company()
    contact_person = fake.name()
    supplier_phone = fake.phone_number()
    supplier_email = fake.email()
    supplier_address = fake.street_address() + ", " + fake.city()
    if len(supplier_address) > 45:
        supplier_address = supplier_address[:45]
    mycursor.execute(sql_supplier, (supplier_name, contact_person, supplier_phone, supplier_email, supplier_address))
    if i % 2 == 0 and i > 0:
        mydb.commit()
        print(f"  {i+1} records committed to Supplier.")
        sys.stdout.flush()
mydb.commit()
print(f"{num_suppliers} suppliers inserted.")
sys.stdout.flush()

# --- 2. Populate 'Customer' table ---
print("Populating Customer table...")
sys.stdout.flush()
num_customers = 500
sql_customer = "INSERT INTO Customer (Customer_id, Name, Email, Phone, Address) VALUES (%s, %s, %s, %s, %s)"
for customer_id in range(1, num_customers + 1):
    name = fake.name()
    email = fake.email()
    phone = str(random.randint(1000000000, 9999999999))
    address = fake.address().replace('\n', ', ')
    mycursor.execute(sql_customer, (customer_id, name, email, phone, address))
    if customer_id % 50 == 0:
        mydb.commit()
        print(f"  {customer_id} records committed to Customer.")
        sys.stdout.flush()
mydb.commit()
print(f"{num_customers} customers inserted.")
sys.stdout.flush()

# --- 3. Populate 'Product' table ---
print("Populating Product table...")
sys.stdout.flush()
num_products = 200
mycursor.execute("SELECT Supplier_id FROM Supplier")
supplier_ids = [row[0] for row in mycursor.fetchall()]
sql_product = "INSERT INTO Product (Product_name, Description, Product_price, Supplier_id) VALUES (%s, %s, %s, %s)"
for i in range(num_products):
    product_name = fake.word().capitalize() + " " + fake.word().capitalize()
    description = fake.sentence(nb_words=10)
    product_price = round(random.uniform(10, 500), 2)
    supplier_id = random.choice(supplier_ids)
    mycursor.execute(sql_product, (product_name, description, product_price, supplier_id))
    if i % 20 == 0 and i > 0:
        mydb.commit()
        print(f"  {i+1} records committed to Product.")
        sys.stdout.flush()
mydb.commit()
print(f"{num_products} products inserted.")
sys.stdout.flush()

# --- 4. Populate 'Orders' table ---
print("Populating Orders table...")
sys.stdout.flush()
num_orders = 1000
mycursor.execute("SELECT Customer_id FROM Customer")
customer_ids = [row[0] for row in mycursor.fetchall()]
sql_order = "INSERT INTO Orders (Customer_id, Order_date, Total_amount) VALUES (%s, %s, %s)"
start_date = datetime.now() - timedelta(days=365)
for i in range(num_orders):
    customer_id = random.choice(customer_ids)
    order_date = fake.date_time_between(start_date, datetime.now())
    total_amount = round(random.uniform(50, 1000), 2)
    mycursor.execute(sql_order, (customer_id, order_date, total_amount))
    if i % 100 == 0 and i > 0:
        mydb.commit()
        print(f"  {i+1} records committed to Orders.")
        sys.stdout.flush()
mydb.commit()
print(f"{num_orders} orders inserted.")
sys.stdout.flush()

# --- 5. Populate 'OrderDetails' table ---
print("Populating OrderDetails table...")
sys.stdout.flush()
mycursor.execute("SELECT Order_id FROM Orders")
order_ids = [row[0] for row in mycursor.fetchall()]
mycursor.execute("SELECT Product_id, Product_price FROM Product")
products_data = {row[0]: row[1] for row in mycursor.fetchall()}
product_ids = list(products_data.keys())
sql_order_details = "INSERT INTO OrderDetails (Order_id, Product_id, Quantity, UnitPrice) VALUES (%s, %s, %s, %s)"
for order_id in order_ids:
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.choice(product_ids)
        quantity = random.randint(1, 10)
        unit_price = products_data[product_id]
        mycursor.execute(sql_order_details, (order_id, product_id, quantity, unit_price))
mydb.commit()
print("Order details inserted.")
sys.stdout.flush()

# --- 6. Populate 'Inventory' table ---
print("Populating Inventory table...")
sys.stdout.flush()
num_logs = 1500
sql_inventory = "INSERT INTO Inventory (Product_id, Quantity_in, Quantity_out, Transaction_date) VALUES (%s, %s, %s, %s)"
for i in range(num_logs):
    product_id = random.choice(product_ids)
    transaction_date = fake.date_time_between(datetime.now() - timedelta(days=365), datetime.now())
    quantity_in = random.randint(0, 100)
    quantity_out = random.randint(0, 50)
    mycursor.execute(sql_inventory, (product_id, quantity_in, quantity_out, transaction_date))
   # The `mydb.commit()` inside the `if` block is repeated and not needed.
# The `print` statement for the total records should be outside the loop.

if i % 100 == 0 and i > 0:
    mydb.commit()
    print(f"  {i+1} records committed to Inventory.")
    sys.stdout.flush()

# This `mydb.commit()` should be outside the loop to commit the last batch
mydb.commit()
print(f"{num_logs} inventory logs inserted.")
sys.stdout.flush()

# --- Close the Connection ---
mycursor.close()
mydb.close()
print("Database connection closed. All tables populated.")
sys.stdout.flush()

Database connected successfully.
Populating Supplier table...
  3 records committed to Supplier.
  5 records committed to Supplier.
  7 records committed to Supplier.
  9 records committed to Supplier.
10 suppliers inserted.
Populating Customer table...
  50 records committed to Customer.
  100 records committed to Customer.
  150 records committed to Customer.
  200 records committed to Customer.
  250 records committed to Customer.
  300 records committed to Customer.
  350 records committed to Customer.
  400 records committed to Customer.
  450 records committed to Customer.
  500 records committed to Customer.
500 customers inserted.
Populating Product table...
  21 records committed to Product.
  41 records committed to Product.
  61 records committed to Product.
  81 records committed to Product.
  101 records committed to Product.
  121 records committed to Product.
  141 records committed to Product.
  161 records committed to Product.
  181 records committed to Product.
200 p